# Material Plotting

In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

The `plot` method of `Material` uses PyVista to quickly visualize fields of the `Material`. Here we will look at how to plot different kinds of fields.

For simple numerical fields, `plot` only requires the name of the field. For example, plot a Voronoi tesselation of grains.

In [ ]:
from materialite import Material
import numpy as np

rng = np.random.default_rng(12345)
num_grains = 3
unique_grains = np.arange(num_grains)
material = Material().create_voronoi(num_regions=num_grains, label="grain", rng=rng)
material.plot("grain")

`plot` defaults include:
- points rendered as voxels with the point at the centroid.
- the `coolwarm` colormap.
- no grid/ruler.
- no voxel edges.

In [ ]:
material.plot("grain", show_grid=True, show_edges=True, colormap="inferno")

Sometimes, a field may be a 1-D numpy array at each point. In this case, a component needs to be passed to `plot` as well. The component refers to the index of the array.

In [ ]:
material = material.create_uniform_field("array", np.arange(3))
material.plot("array", component=2)

Many mechanical models will create stress and/or strain fields in the `Material` that consist of symmetric second-order tensors. Here, `component` refers to the tensor component in Voigt notation $(0 \rightarrow xx,\ 1 \rightarrow yy,\ 2 \rightarrow zz,\ 3 \rightarrow yz,\ 4 \rightarrow xz,\ 5 \rightarrow xy)$

In [ ]:
from materialite import Order2SymmetricTensor

stresses = np.zeros((num_grains, 6))
stresses[:, 0] = [1000, 10, 100]
grainwise_stresses = Order2SymmetricTensor.from_stress_voigt(stresses)
material = material.create_regional_fields(
    region_label="grain",
    regional_fields={"grain": unique_grains, "stress": grainwise_stresses},
)
material.plot("stress", component=0)

It is possible to modify the bounds on the data that is plotted. For example, we can increase the lower bound to better visualize the stress distribution in stiffer grains.

In [ ]:
material.plot("stress", component=0, color_lims=(500, 1000))

`Scalar`s and `Vector`s from the Materialite tensor library can also be plotted. `Scalar` fields are handled like numerical fields, and `Vector` fields are handled like numpy arrays.

`plot` can also be used to make an inverse pole figure map if there is a field containing RGB values corresponding to the IPF. Materialite has a utility function to create this field given a field of `Orientation`s, a unit cell type, and a reference direction.

In [ ]:
from materialite import add_ipf_colors_field

material = material.assign_random_orientations(
    region_label="grain", orientation_label="orientation", rng=rng
)
material = add_ipf_colors_field(
    material,
    orientation_label="orientation",
    specimen_frame_direction=[0, 0, 1],
    unit_cell="cubic",
    ipf_color_label="ipf_color",
)
material.plot("ipf_color", kind="ipf_map")